In [13]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import pandas as pd
import numpy as np 
import openpyxl
import sklearn.linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier  
from sklearn.ensemble import RandomForestRegressor  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re
import statsmodels.api as sm
import os
from Fbref_alex.fonctions_preprocess import preprocess_intitial
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

df_stats = pd.read_csv('Fbref_alex\\SOCCER_241223.csv')
mapping_equipe = {
        'Nimes': 'Nîmes',
        'Paris S-G': 'Paris Saint Germain',
        'Saint Etienne': 'Saint-Étienne'
    }

df_stats = preprocess_intitial(df_stats, mapping_equipe)
df_stats[['GF', 'GA']] = df_stats[['GF', 'GA']].astype(float).astype(int)
df_stats.insert(0, 'Victoire',  df_stats['Result'].apply(lambda x: 1 if x == 'W' else 0))
df_stats.insert(1, 'Défaite', df_stats['Result'].apply(lambda x: 1 if x == 'L' else 0))
df_stats.insert(2, 'Nul', df_stats['Result'].apply(lambda x: 1 if x == 'D' else 0))
df_stats.drop(['Result'], axis=1, inplace=True)

def fonction_accueil(canvas, bouton_accueil):
    canvas.get_tk_widget().destroy()
    bouton_accueil.destroy()
    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_intro.place(in_=window, anchor="w", relx=0.47, rely=0.1)

    label_choix = tk.Label(frame_menus, text="Que voulez vous faire?")
    label_choix.grid(row=0, column=0, padx=10, pady=10)

    choix_combobox = ttk.Combobox(frame_menus, values=['Visualisez des stats', 'Prédire un résultat'], state="readonly")
    choix_combobox.set("Sélectionnez un choix")
    choix_combobox.grid(row=0, column=1, padx=10, pady=10)
    choix_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix(label_choix, choix_combobox, e))

    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)


def fonction_choix(label_choix, choix_combobox, event):
    sélection = choix_combobox.get()
    if sélection == 'Visualisez des stats':
        fonction_choix_saison(label_choix, choix_combobox, event = None)
    elif sélection == 'Prédire un résultat':
        fonction_predictions(label_choix, choix_combobox, event = None)

def fonction_choix_saison(label_choix, choix_combobox, event):
    label_choix.destroy()
    choix_combobox.destroy()

    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)
    label_choix_saison = tk.Label(frame_menus, text="Pour quelle saison?")
    label_choix_saison.grid(row=0, column=0, padx=10, pady=10)

    choix_saisons = df_stats['Saison'].unique().tolist()
    choix_saisons =  choix_saisons[:] + ['Toutes les saisons'] 
    saison_combobox = ttk.Combobox(frame_menus, values=choix_saisons, state="readonly")
    saison_combobox.set("Sélectionnez une saison")
    saison_combobox.grid(row=0, column=1, padx=10, pady=10)
    saison_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix_equipe(label_choix_saison, saison_combobox, e))


def fonction_choix_equipe(label_choix_saison, saison_combobox, e):
    saison = saison_combobox.get()
    label_choix_saison.destroy()
    saison_combobox.destroy()

    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)

    label_choix_equipe = tk.Label(frame_menus, text="Pour quelle équipe?")
    label_choix_equipe.grid(row=0, column=0, padx=10, pady=10)
    
    if saison == 'Toutes les saisons':
        choix_equipe = df_stats['Team'].unique().tolist()
    else:
        choix_equipe = df_stats[df_stats['Saison'] == saison]['Team'].unique().tolist()
    choix_equipe =  choix_equipe[:] + ['Toutes les équipes']
    equipe_combobox = ttk.Combobox(frame_menus, values=choix_equipe, state="readonly")
    equipe_combobox.set("Sélectionnez une équipe")
    equipe_combobox.grid(row=0, column=1, padx=10, pady=10)
    equipe_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix_stats(label_choix_equipe, equipe_combobox, saison, e))


def fonction_choix_stats(label_choix_equipe, equipe_combobox, saison, e):
    equipe = equipe_combobox.get()
    label_choix_equipe.destroy()
    equipe_combobox.destroy()
    
    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)

    label_choix_stats = tk.Label(frame_menus, text="Pour quelle stat?")
    label_choix_stats.grid(row=0, column=0, padx=10, pady=10)
    

    choix_stats =  [elem for elem in df_stats.columns if elem not in ['Saison', 'Team', 'Opponent', 'DateTime', 'Unnamed: 0', 'Comp', 'Round', 
                                                                      'Day', 'Venue', 'Attendance', 'Captain', 'Formation', 'Referee', 'Match Report', 
                                                                      'Notes', 'MatchID']]
    stats_combobox = ttk.Combobox(frame_menus, values=choix_stats, state="readonly")
    stats_combobox.set("Sélectionnez une stat")
    stats_combobox.grid(row=0, column=1, padx=10, pady=10)
    stats_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix_filtre(label_choix_stats, stats_combobox, saison, equipe, e))


def fonction_choix_filtre(label_choix_stats, stats_combobox, saison, equipe, e):
    stats = stats_combobox.get()
    label_choix_stats.destroy()
    stats_combobox.destroy()
    
    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)

    label_choix_filtre = tk.Label(frame_menus, text="Avec quel filtre?")
    label_choix_filtre.grid(row=0, column=0, padx=10, pady=10)
    

    choix_filtre =  [elem for elem in df_stats.columns if elem in ['Saison', 'Team', 'Round', 'Opponent', 'Venue', 'Formation']]
    filtre_combobox = ttk.Combobox(frame_menus, values=choix_filtre, state="readonly")
    filtre_combobox.set("Sélectionnez un filtre")
    filtre_combobox.grid(row=0, column=1, padx=10, pady=10)
    filtre_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix_type(label_choix_filtre, filtre_combobox, saison, equipe, stats, e))

def fonction_choix_type(label_choix_filtre, filtre_combobox, saison, equipe, stats, e):
    filtre = filtre_combobox.get()
    label_choix_filtre.destroy()
    filtre_combobox.destroy()
    
    
    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)

    label_choix_type = tk.Label(frame_menus, text="Quel type de stat?")
    label_choix_type.grid(row=0, column=0, padx=10, pady=10)
    

    choix_type =  ['Somme', 'Moyenne', 'Value Counts']
    type_combobox = ttk.Combobox(frame_menus, values=choix_type, state="readonly")
    type_combobox.set("Sélectionnez un type de stats")
    type_combobox.grid(row=0, column=1, padx=10, pady=10)
    type_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix_graph(label_choix_type, type_combobox, saison, equipe, stats, filtre, e))

def fonction_choix_graph(label_choix_type, type_combobox, saison, equipe, stats, filtre, e):
    type = type_combobox.get()
    label_choix_type.destroy()
    type_combobox.destroy()
    
    frame_menus = tk.Frame(window)
    frame_menus.pack(expand=True, anchor="w")
    frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)

    label_choix_graph = tk.Label(frame_menus, text="Quel type de graph?")
    label_choix_graph.grid(row=0, column=0, padx=10, pady=10)
    

    choix_graph =  ['Linéaire', 'Barres']
    graph_combobox = ttk.Combobox(frame_menus, values=choix_graph, state="readonly")
    graph_combobox.set("Sélectionnez un graph")
    graph_combobox.grid(row=0, column=1, padx=10, pady=10)
    graph_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_affichage(label_choix_graph, graph_combobox, saison, equipe, stats, filtre, type, e))

def fonction_affichage(label_choix_graph, graph_combobox, saison, equipe, stats, filtre, type, e):
    graph = graph_combobox.get()
    if saison == 'Toutes les saisons':
        if equipe == 'Toutes les équipes':
            df_plot = df_stats
        else:   
            df_plot = df_stats[(df_stats['Team'] == str(equipe))]
    else:
        if equipe == 'Toutes les équipes':
            df_plot = df_stats[(df_stats['Saison'] == str(saison))]
        else:
            df_plot = df_stats[(df_stats['Team'] == str(equipe)) & (df_stats['Saison'] == str(saison))]
    if type == 'Somme':
        if graph == 'Linéaire':
            plot = df_plot.groupby(str(filtre))[str(stats)].sum().reset_index()

            figure = Figure(figsize=(10, 6), dpi=100)
            subplot = figure.add_subplot(1, 1, 1)


            subplot.plot(plot[str(filtre)], plot[str(stats)], marker='o', linestyle='-')
            subplot.set_title('Évolution de ' + str(stats) + ' de ' + str(equipe) + ' en fonction de ' + str(filtre))
            subplot.set_xlabel(str(filtre))
            subplot.set_ylabel(str(stats))
            subplot.grid(True)

            canvas = FigureCanvasTkAgg(figure, master=window)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        if graph == 'Barres':
            plot = df_plot.groupby(str(filtre))[str(stats)].sum().reset_index()

            figure = Figure(figsize=(10, 6), dpi=100)
            subplot = figure.add_subplot(1, 1, 1)


            subplot.bar(plot[str(filtre)], plot[str(stats)], color = 'blue')
            subplot.set_title('Nombre de ' + str(stats) + ' de ' + str(equipe) + ' en fonction de ' + str(filtre))
            subplot.set_xlabel(str(filtre))
            subplot.set_ylabel(str(stats))
            subplot.grid(True)

            canvas = FigureCanvasTkAgg(figure, master=window)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    elif type == 'Moyenne':
        if graph == 'Linéaire':
            plot = df_plot.groupby(str(filtre))[str(stats)].mean().reset_index()

            figure = Figure(figsize=(10, 6), dpi=100)
            subplot = figure.add_subplot(1, 1, 1)


            subplot.plot(plot[str(filtre)], plot[str(stats)], marker='o', linestyle='-')
            subplot.set_title('Évolution de ' + str(stats) + ' de ' + str(equipe) + ' en fonction de ' + str(filtre))
            subplot.set_xlabel(str(filtre))
            subplot.set_ylabel(str(stats))
            subplot.grid(True)

            canvas = FigureCanvasTkAgg(figure, master=window)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        if graph == 'Barres':
            plot = df_plot.groupby(str(filtre))[str(stats)].mean().reset_index()

            figure = Figure(figsize=(10, 6), dpi=100)
            subplot = figure.add_subplot(1, 1, 1)


            subplot.bar(plot[str(filtre)], plot[str(stats)], color = 'blue')
            subplot.set_title('Nombre de ' + str(stats) + ' de ' + str(equipe) + ' en fonction de ' + str(filtre))
            subplot.set_xlabel(str(filtre))
            subplot.set_ylabel(str(stats))
            subplot.grid(True)

            canvas = FigureCanvasTkAgg(figure, master=window)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    else:
        if graph == 'Linéaire':
            plot = df_plot.groupby(str(filtre))[str(stats)].value_counts().reset_index()

            figure = Figure(figsize=(10, 6), dpi=100)
            subplot = figure.add_subplot(1, 1, 1)


            subplot.plot(plot[str(filtre)], plot[str(stats)], marker='o', linestyle='-')
            subplot.set_title('Évolution de ' + str(stats) + ' de ' + str(equipe) + ' en fonction de ' + str(filtre))
            subplot.set_xlabel(str(filtre))
            subplot.set_ylabel(str(stats))
            subplot.grid(True)

            canvas = FigureCanvasTkAgg(figure, master=window)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        if graph == 'Barres':
            plot = df_plot.groupby(str(filtre))[str(stats)].value_counts().reset_index()

            figure = Figure(figsize=(10, 6), dpi=100)
            subplot = figure.add_subplot(1, 1, 1)


            subplot.bar(plot[str(filtre)], plot[str(stats)], color = 'blue')
            subplot.set_title('Nombre de ' + str(stats) + ' de ' + str(equipe) + ' en fonction de ' + str(filtre))
            subplot.set_xlabel(str(filtre))
            subplot.set_ylabel(str(stats))
            subplot.grid(True)

            canvas = FigureCanvasTkAgg(figure, master=window)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

    bouton_accueil = ttk.Button(window, text="Revenir à la page d'accueil", command=lambda: fonction_accueil(canvas, bouton_accueil))
    bouton_accueil.pack(side=tk.BOTTOM)


def fonction_predictions(label_choix, choix_combobox, event):
    label_choix.destroy()
    choix_combobox.destroy()

window = tk.Tk()
window.title("Choix de l'utilisateur")
window.attributes('-fullscreen', True)

image_intro = Image.open('Logos\\Logo_Ligue_1_Uber_Eats_2022.png')
image_intro = image_intro.resize((130, 180), Image.LANCZOS)
photo_intro = ImageTk.PhotoImage(image_intro)
frame_intro = tk.Frame(window)
frame_intro.pack(expand=True, anchor="w")
label_image_intro = tk.Label(frame_intro, image=photo_intro)
label_image_intro.image = photo_intro
label_image_intro.grid(row=0, column=0)


frame_menus = tk.Frame(window)
frame_menus.pack(expand=True, anchor="w")
frame_intro.place(in_=window, anchor="w", relx=0.47, rely=0.1)

label_choix = tk.Label(frame_menus, text="Que voulez vous faire?")
label_choix.grid(row=0, column=0, padx=10, pady=10)

choix_combobox = ttk.Combobox(frame_menus, values=['Visualisez des stats', 'Prédire un résultat'], state="readonly")
choix_combobox.set("Sélectionnez un choix")
choix_combobox.grid(row=0, column=1, padx=10, pady=10)
choix_combobox.bind("<<ComboboxSelected>>", lambda e: fonction_choix(label_choix, choix_combobox, e))

frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)


window.mainloop()



In [41]:
df_stats = pd.read_csv('Fbref_alex\\SOCCER_241223.csv')
mapping_equipe = {
        'Nimes': 'Nîmes',
        'Paris S-G': 'Paris Saint Germain',
        'Saint Etienne': 'Saint-Étienne'
    }

df_stats = preprocess_intitial(df_stats, mapping_equipe)
df_stats[['GF', 'GA']] = df_stats[['GF', 'GA']].astype(float).astype(int)
df_stats.groupby('Saison')['GF'].sum()


Saison
2018-2019     972
2019-2020     704
2020-2021    1049
2021-2022    1067
2022-2023    1067
2023-2024     382
Name: GF, dtype: int32